# Maryland Child and Dependent Care Credit

Maryland's Child and Dependent Care Tax Credit (CDCC) provides up to 32% of the federal CDCC, depending on income and filing status.

## Examples

Consider a single parent in Maryland with one child and \$8,000 child care expenses (the maximum for one child in 2021).

In [7]:
from openfisca_us import IndividualSim
import pandas as pd
import plotly.express as px


def make_cdcc(adults, children):
    sim = IndividualSim(year=2022)
    sim.add_person(name="head", is_tax_unit_head=True)
    members = ["head"]
    if adults == 2:
        sim.add_person(name="spouse", is_tax_unit_spouse=True)
        members += ["spouse"]
    for i in range(children):
        child = "child{}".format(i)
        sim.add_person(name=child, age=5)
        members += [child]
    sim.add_tax_unit(name="tax_unit", members=members, tax_unit_childcare_expenses=16_000)
    sim.add_spm_unit(name="spm_unit", members=members)
    sim.add_household(name="household", members=members, state_code="MD")
    print(sim.calc("filing_status"))
    sim.vary("employment_income", max=160_000, step=100)
    return pd.DataFrame(
        dict(
            employment_income=sim.calc("employment_income")[0],
            md_cdcc=sim.calc("md_cdcc")[
                0
            ].round(),
            cdcc_mtr=-sim.deriv(
                "md_cdcc",
                "employment_income",
                wrt_target="head",
            ),
            adults=adults,
            children=children,
        )
    )


# Make a table of state taxes for different numbers of adults and children.
l = []
for adults in range(1, 3):
    for children in range(1, 3):
        l.append(make_cdcc(adults, children))

df = pd.concat(l)

LABELS = dict(
    employment_income="Employment income",
    md_cdcc="MD CDCC",
    scb_mtr="MD CDCC Marginal Tax Rate",
    adults="Adults",
    children="Children",
)

fig = px.line(
    df,
    "employment_income",
    "md_cdcc",
    color="children",
    animation_frame="adults",
    labels=LABELS,
    title="Maryland CDCC",
)
fig.update_layout(xaxis_tickformat="$,", yaxis_tickformat="$,")
fig.show()

['HEAD_OF_HOUSEHOLD']
['HEAD_OF_HOUSEHOLD']
['JOINT']
['JOINT']


The credit phases out at 10% of income where it phases out smoothly.

In [10]:
fig = px.line(
    df,
    "employment_income",
    "scb_mtr",
    color="adults",
    labels=LABELS,
    title="Massachusetts Senior Circuit Breaker Credit marginal tax rate",
)
fig.update_layout(
    xaxis_tickformat="$,", yaxis_tickformat=".1%", yaxis_range=[0, 0.2]
)
fig.show()